# Date Standardization

Written dates can come in many formats, it may be helpful to be able to standardize them for further processing.

## In this notebook
1. Implement a vanilla Seq2Seq model using a LSTM.
2. Implement the dot product variant of Luong attention.
3. Plot attention matrix.

## 0. Imports

In [1]:
from datetime import datetime
import random
from pathlib import Path

In [3]:
import numpy as np
import pandas as pd

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [6, 8]

In [7]:
from keras import models, layers, callbacks, utils

## 1. Datasets

Since it's very easy to generate dates at random, we're going to build the dataset rather than retrieve an existing one.

It will be saved for reuse in successive notebook executions.

This code is a reworking of one originally proposed by Andrew Ng.

In [32]:
class DateGenerator:
    """
    Generate a date string dataset with dates in a random string format and the standard format.
    """

    std_format = '%Y-%m-%d'
    formats = [
        '%Y-%m-%d',
        '%d-%m-%Y',
        '%m-%d-%Y',

        '%Y/%m/%d',
        '%d/%m/%Y',
        '%m/%d/%Y',

        '%b %d, %Y',
        '%b %d, %y',
        '%B %d, %Y',

        #'%A %B %-d, %Y',  # day of week, full month, day, year
        #'%A %B %-d %Y',

        '%b %d %Y',  # abreviated month, day, year

        '%B %d %Y',  # full month, day, year
    ]

    def random_datetime(self):
        """Get a random datetime object."""
        rand_year = random.randint(1900, 2100)
        rand_day = random.randint(1, 31)
        rand_month = random.randint(1, 12)
        try:
            return datetime(rand_year, rand_month, rand_day)
        except ValueError:
            return self.random_datetime()
    
    def get_sample(self):
        """
        Get a random sample.
        Returns the date in (standard_format, random_format, format_index)
        """
        date = self.random_datetime()
        rand_format = random.choice(self.formats)
        format_index = self.formats.index(rand_format)
        std_str = date.strftime(self.std_format)
        rand_str = date.strftime(rand_format)
        return (std_str, rand_str.lower(), format_index)
    
    def generate_df(self, n: int):
        """Generate a df with `n` samples."""
        return pd.DataFrame(
            [self.get_sample() for _ in range(n)],
            columns=['output', 'input', 'format']
        )

In [34]:
_DATA_PATH = Path('data')
if not _DATA_PATH.exists():
    _DATA_PATH.mkdir(parents=True)

date_generator = DateGenerator()

if not (_DATA_PATH / 'train.csv').exists():
    train_df = date_generator.generate_df(100000)
    train_df.to_csv(_DATA_PATH / 'train.csv', index=False)
else:
    train_df = pd.read_csv(_DATA_PATH / 'train.csv')

if not (_DATA_PATH / 'val.csv').exists():
    val_df = date_generator.generate_df(10000)
    val_df.to_csv(_DATA_PATH / 'val.csv', index=False)
else:
    val_df = pd.read_csv(_DATA_PATH / 'val.csv')

if not (_DATA_PATH / 'test.csv').exists():
    test_df = date_generator.generate_df(10000)
    test_df.to_csv(_DATA_PATH / 'test.csv', index=False)
else:
    test_df = pd.read_csv(_DATA_PATH / 'test.csv')

In [36]:
train_df.head()

,output,input,format
0,1914-09-21,21-09-1914,1
1,2078-04-01,01-04-2078,1
2,1998-07-04,07/04/1998,5
3,2002-01-13,01/13/2002,5
4,1994-09-02,02/09/1994,4


In [38]:
# If the format is 7, then the year is only 2 digits long. This makes it difficult to predict a 4-digit year
print("best possible score", (len(test_df)-np.sum(test_df['format']==7))/len(test_df))

best possible score 0.9034


### Load data and create character -> int mapping

The letter -> id transformation will be performed using a dictionary that we need to build.

In [59]:
def create_vocab():
    tokens = ['<pad>', '<unk>', '<sos>', '<eos>'] + sorted(list(set(list((''.join(train_df['input'])+''.join(train_df['output'])).lower()))))
    print("Tokens:",tokens)
    tok_to_int = {c: i for i, c in enumerate(tokens)}
    int_to_tok = [c for c, i in tok_to_int.items()]
    assert tok_to_int['<pad>'] == 0 # by convention
    assert tok_to_int['<unk>'] == 1 # by convention
    assert tok_to_int['<sos>'] == 2 # start of sequence
    assert tok_to_int['<eos>'] == 3 # end of sequence
    return tok_to_int, int_to_tok

In [69]:
tok_to_int, int_to_tok = create_vocab()
_VOCAB_SIZE = len(tok_to_int)
print("\nTaille du vocabulaire :", _VOCAB_SIZE)

Tokens: ['<pad>', '<unk>', '<sos>', '<eos>', ' ', ',', '-', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y']

Taille du vocabulaire : 39


### Define a dataset

Input (X): 

    encoder input: Noisy date text.i.e.
                   `Saturday December 23, 1834`
    
    decoder input: (the teacher) the standard date text without the last timestep,
                   include <sos> excludes <eos>. 
                   This is only needed when we are training with "teacher forcing".
                   Otherwise the decoder makes predictions on its own without knowing
                   what it the correct answer was for the previous timestep.
                   i.e. <sos>1834-12-23
    
Output (y): Standard date text without the first timestep, include <eos>, excludes <sos>
                   i.e. i.e. 1834-12-23<eos>
    
The following dataset code is specific to pytorch and may seem a bit convoluted. It just maps an integer index to a single training sample, as described above.

In [49]:
# for encoder input the padding is 'pre', <sos> and <eos> are not added
# for output <eos> is added at the end, followed by padding if necessary 
def define_dataset(data, max_len, padding='pre'):
    result = np.zeros((len(data), max_len), dtype=int)
    for i, d in enumerate(data):
        d = [tok_to_int[i] for i in list(d)]
        assert len(d) <= max_len
        if padding=='post': # for y
            result[i,:(len(d)+1)] = d + [tok_to_int['<eos>']]
        else: # for X
            result[i,-len(d):] = d
    assert np.max(result) < _VOCAB_SIZE
    return result

In [89]:
# Define X
_INPUT_LENGTH = max(len(d) for d in train_df['input'])
X_train = define_dataset(train_df['input'], _INPUT_LENGTH, padding='pre')
X_val = define_dataset(val_df['input'], _INPUT_LENGTH, padding='pre')
X_test = define_dataset(test_df['input'], _INPUT_LENGTH, padding='pre')
X_train[0]
print("Exemple d'entrée encodeur :", X_train[0])

Exemple d'entrée encodeur : [ 0  0  0  0  0  0  0  0 10  9  6  8 17  6  9 17  9 12]


In [99]:
# Define y
_OUTPUT_LENGTH = max(len(d) for d in train_df['output']) + 1
y_train = define_dataset(train_df['output'], _INPUT_LENGTH, padding='post')
y_val = define_dataset(val_df['output'], _INPUT_LENGTH, padding='post')
y_test = define_dataset(test_df['output'], _INPUT_LENGTH, padding='post')
print("Exemple de sortie  décodeur :", y_train[0])

Exemple de sortie  décodeur : [ 9 17  9 12  6  8 17  6 10  9  3  0  0  0  0  0  0  0]


In [101]:
def create_teacher_data(y_data, max_len):
    teacher_data = np.zeros((len(y_data), max_len), dtype=int)

    for i, seq in enumerate(y_data):
        seq_without_eos = seq[seq > 0][:-1] 
        teacher_data[i, :len(seq_without_eos) + 1] = [tok_to_int['<sos>']] + list(seq_without_eos)
    
    return teacher_data

In [111]:
# Define decoder input (teacher) : add sos at the top + copy the output minus the last element
teacher_train = create_teacher_data(y_train, _OUTPUT_LENGTH)
teacher_val = create_teacher_data(y_val, _OUTPUT_LENGTH)
teacher_test = create_teacher_data(y_test, _OUTPUT_LENGTH)
print("Exemple d'entrée teacher forcing :", teacher_train[0])

output_tokens = [int_to_tok[i] for i in teacher_train[0]]

output_text = ''.join(output_tokens)
print(output_text)

Exemple d'entrée teacher forcing : [ 2  9 17  9 12  6  8 17  6 10  9]
<sos>1914-09-21


# 2. Vanilla RNN/LSTM
![Luong Figure 1.](img/luong_seq2seq.png)

### Implementation:
1. Encode the input text, you will get 1 vector for each step in the input.
2. Keep the RNN's hidden state from the last time step, call this $h_t$. In the picture, these are the two arrows between the blue and red blocks.
3. Initialize the decoder's hidden state with $h_t$.
4. Let the decoder make predictions one step at a time. This will modify $h_t$.
5. Feed the last prediction and $h_t$ as the next step of the decoder.
6. Stop when you reach the "<eos\>" marker.

In [ ]:
# set model config
# this is a pretty small network, any bigger and the model can easy solve this problem.
# I am purposefully trying to make it difficult for the vanilla model to "solve" this problem.

_EMBEDDING_SIZE = 32
_RNN_SIZE = 32
_DROPOUT_RATE = 0.5

In [ ]:
# For saving models
_MODEL_PATH = Path('models')
if not _MODEL_PATH.exists():
    _MODEL_PATH.mkdir(parents=True)

### Seq2Seq Model

Keras implementation of a Seq2Seq model using an embedding layer followed a RNN layer.

I have commented on the shape of the output tensor for each line.

```
B = batch size
T = time steps
e = embedding dimension
```

In [ ]:
def build_encoder():
    inputs = layers.Input(shape=(_INPUT_LENGTH,), name="encInput")
    h = layers.Embedding(input_dim=_VOCAB_SIZE, output_dim=_EMBEDDING_SIZE)(inputs)
    encoder_outputs, memory_state, carry_state = layers.LSTM(_RNN_SIZE, return_sequences=True, return_state=True,
                                                             dropout=_DROPOUT_RATE, recurrent_dropout=_DROPOUT_RATE)(h)
    encoder_context = [memory_state, carry_state]
    return models.Model(inputs, [encoder_outputs, encoder_context], name="encoder")

In [ ]:
encoder = build_encoder()
utils.plot_model(encoder,
                 show_shapes=True,
                #show_dtype=True,
                show_layer_names=True,
                rankdir="TB",
                expand_nested=True,
                dpi=200,
                show_layer_activations=True,
                show_trainable=True)

In [ ]:
def build_decoder():
    teacher_inputs = layers.Input(shape=(_OUTPUT_LENGTH,), name="decInput")
    context_input = [layers.Input(shape=(_RNN_SIZE,), name="ctxInputH"),
                     layers.Input(shape=(_RNN_SIZE,), name="ctxInputC")]

    # Meme principe que l'encoder
    # Une couche d'embedding
    ... = layers.Embedding(input_dim=..., output_dim=...)(...)
    # Une couche de RNN dont l'état initial est context_input
    decoder_outputs, memory_state, carry_state = layers.LSTM(_RNN_SIZE, return_sequences=..., return_state=...,
                                                            dropout=_DROPOUT_RATE, recurrent_dropout=_DROPOUT_RATE)(..., initial_state=...)
    context = [memory_state, carry_state]
    # Une couche Dense pour prédire la bonne lettre
    outputs = layers.Dense(..., activation='softmax')(...)
    return models.Model([teacher_inputs, context_input], [outputs, context], name="decoder")

In [ ]:
decoder = build_decoder()
utils.plot_model(decoder,
                 show_shapes=True,
                #show_dtype=True,
                show_layer_names=True,
                rankdir="TB",
                expand_nested=True,
                dpi=200,
                show_layer_activations=True,
                show_trainable=True)

In [ ]:
def build_seq2seq(encoder, decoder):
    # Il est maintenant possible de construire le Seq2Seq en connectant l'encodeur au decodeur
    
    enc_inputs = layers.Input(shape=(_INPUT_LENGTH,), name="encInput")
    teacher_inputs = layers.Input(shape=(_OUTPUT_LENGTH,), name="teacherInput")

    # Appel de l'encoder
    ..., ... = encoder(...)

    # Appel du décodeur
    ..., ... = decoder([..., ...])

    # Construction du modèle 
    # L'entrée est l'entrée de l'encodeur et l'entrée du décodeur (le teacher)
    # La sortie est la sequence de sortie du décodeur
    return models.Model([enc_inputs, teacher_inputs], [...])

In [ ]:
seq2seq = build_seq2seq(encoder, decoder)
utils.plot_model(seq2seq,
                 show_shapes=True,
                #show_dtype=True,
                show_layer_names=True,
                rankdir="TB",
                expand_nested=True,
                dpi=200,
                show_layer_activations=True,
                show_trainable=True)

## Train teacher seq2seq

In [ ]:
# Entrainement du modèle seq2seq - rien de particulier si ce n'est que c'est un peu long
# Profittez-en pour réfléchir à la manière dont vous allez utiliser ce modèle pour prédire
if not ((_MODEL_PATH / "encoder.keras").exists() and (_MODEL_PATH / "decoder.keras").exists() and (_MODEL_PATH / "seq2seq.keras").exists()):
    seq2seq.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    hist = seq2seq.fit([X_train, teacher_train], y_train,
          validation_data=([X_val, teacher_val], y_val),
          batch_size=128,
          epochs=500,
          callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
    )
    pd.DataFrame.from_dict(hist.history).plot()
    encoder.save(_MODEL_PATH / "encoder.keras")
    decoder.save(_MODEL_PATH / "decoder.keras")
    seq2seq.save(_MODEL_PATH / "seq2seq.keras")
else:
    encoder = models.load_model(_MODEL_PATH / "encoder.keras")
    decoder = models.load_model(_MODEL_PATH / "decoder.keras")
    seq2seq = models.load_model(_MODEL_PATH / "seq2seq.keras")

### Predict with the trained model

In [ ]:
# Pour transformer la liste d'id en une chaine de caractères
def decode(pred):
    return ''.join([int_to_tok[int(id)] for id in pred if id not in [0,2]])

In [ ]:
def predict(seq2seq, X):    
    ...

In [ ]:
pred = predict(seq2seq, X_test)

In [ ]:
# Vérification sur une valeur
decode(X_test[0]), decode(y_test[0]), decode(pred[0])

In [ ]:
# On peut évaluer le modèle sur sa capacité à reconnaitre chaque caractère
# même si cela à peu de sens
from sklearn.metrics import classification_report

print(classification_report(y_test.ravel(), pred.ravel()))

In [ ]:
# Il est bien plus pertinent de regarder si la chaine prédite est égale à la chaine à prédire
def score(y_true, y_pred):
    # renvoie le pourcentage de date correctement prédite
    return np.round(100*(len(y_true)-inc)/len(y_true),2)

In [ ]:
# Votre resultat devrait être entre 70 et 80 %
score(y_test, pred)

# 3. Seq2Seq w/ Luong Attention



Les attentions sont décrites dans le papier suivant (https://arxiv.org/pdf/1508.04025.pdf) qui est une mise en œuvre de l'attention de Luong. Il existe de nombreux modèles d'attention mais celle-ci est la plus simple et est suffisante pour ce TP.

![Luong Figure 2.](img/luong_attn.png)

### Justification du mécanisme d'Attention
* Un seul vecteur de longueur fixe ne peut pas stocker une quantité infinie d'information.
* Le défi du Seq2Seq est que beaucoup d'informations sont perdues entre l'encodeur et le décodeur. En effet, un seul vecteur de longueur fixe (le dernier état caché de l'encodeur) est transmis au décodeur. Cela oblige le décodeur à traduire le message codé avec des informations dégradées.
* Une meilleure approche consisterait à transmettre l'intégralité de la séquence codée à chacune des étapes du décodeur, ce qui garantit l'absence de perte d'informations. En pratique, il s'agit de la séquence d'états cachés du codeur qui pourrait être accessible à chacune des étapes du décodeur.
* Le mécanisme d'attention permet de « se concentrer » sur un élément particulier du contexte à partir d'une requête. Ce mécanisme est couramment utilisé dans la traduction automatique neuronale (NMT). Dans la traduction automatique neuronale, le « contexte » est défini par le codeur.

### Mise en oeuvre
1. Encodage du texte d'entrée pour récupérer 1 vecteur pour chaque étape de l'entrée. Ce tenseur que l'on appelera value aura la forme (None, _INPUT_LENGTH, _RNN_SIZE)
2. De la même manière on récupère 1 vecteur pour chaque étape du décodeur. Ce tenseur que l'on appelera query aura la forme (None, _OUTPUT_LENGTH, _RNN_SIZE)
3. Pour chacune des paires (query, value), nous allons effectuer le produit vectoriel des deux vecteurs pour obtenir un tenseur de la forme (None, _OUTPUT_LENGTH, _INPUTS_LENGTH). Il s'agit des scores bruts.
4. L'étape suivante est très simple puisqu'il s'agit de normaliser cette matrice à l'aide d'un 'softmax' pour obtenir les scores d'attention
5. Il s'agit maintenant de faire le produit vectoriel de chacune des valeurs issues de l'encodeur (value) par son score correspondant et ce, pour chacune des étapes du décodeur. On obtient un tenseur de la forme (None, _OUTPUT_LENGTH, _RNN_SIZE)
6. On peut maintenant concaténer ce tenseur avec la sortie query du décodeur pour prédire le token de sortie

In [ ]:
# L'encodeur n'est pas modifié

In [ ]:
# Implémentation des étape 3, 4 et 5
def attention(query, value, mode='dot'):
    # 1. value vient de l'encodeur - shape : None, _OUTPUT_LENGTH, _RNN_SIZE
    # 2. query vient du décodeur - shape : None, _INPUT_LENGTH, _RNN_SIZE
    
    # 3. la fonction mathématique dot (produit vectoriel) permet de calculer une similarité entre 2 vecteurs - attention à bien utiliser les bon 'axes'
    attention = layers.dot(...)
    assert attention.shape == (None, _OUTPUT_LENGTH, _INPUT_LENGTH)

    # 4. normalisation des scores: uniquement un couche d'Activation='softmax'
    attention_scores = layers....
    assert attention_scores.shape == (None, _OUTPUT_LENGTH, _INPUT_LENGTH)

    # 5. calcul du contexte d'attention - attention à bien utiliser les bon 'axes'
    attention_context = layers.dot(...)
    assert attention_context.shape == (None, _OUTPUT_LENGTH, _RNN_SIZE)
    
    return attention_context, attention_scores

In [ ]:
# Le decodeur est un peu modifié pour inclure
# --> le calcul de l'attention par appel de la fonction précédente
# --> et effectuer la concaténation entre le vecteur de contexte et la sortie de la couche RNN du décodeur (étape 6)
def build_decoder_attention():
    teacher_inputs = layers.Input(shape=(_OUTPUT_LENGTH,), name="decInput")
    context_input = [layers.Input(shape=(_RNN_SIZE,), name="ctxInputH"),
                     layers.Input(shape=(_RNN_SIZE,), name="ctxInputC")]
    encoder_outputs = layers.Input(shape=(_INPUT_LENGTH, _RNN_SIZE), name="EncoderOuputs")

    h = layers.Embedding(...)(...)
    decoder_outputs, memory_state, carry_state = layers.LSTM(...)(..., initial_state=...)
    decoder_context = [memory_state, carry_state]

    attention_context, attention_scores = attention(..., ...)
    
    decoder_outputs = layers.concatenate([..., ...])
    outputs = layers.Dense(..., activation='...')(...)
    
    return models.Model([encoder_outputs, teacher_inputs, context_input], [outputs, decoder_context, attention_scores], name="decoder")

In [ ]:
decoder_attention = build_decoder_attention()
utils.plot_model(decoder_attention,
                 show_shapes=True,
                #show_dtype=True,
                show_layer_names=True,
                rankdir="TB",
                expand_nested=True,
                dpi=200,
                show_layer_activations=True,
                show_trainable=True)

In [ ]:
# Reste comme précédemment à assembler l'encodeur est le décodeur
def build_seq2seq_attention(encoder, decoder):
    enc_inputs = layers.Input(shape=(_INPUT_LENGTH,), name="encInput")
    enc_outputs, enc_context = encoder(enc_inputs)
    
    teacher_inputs = layers.Input(shape=(_OUTPUT_LENGTH,), name="teacherInput")
    dec_sequence_output, _, _ = decoder_attention([enc_outputs, teacher_inputs, enc_context])
    return models.Model([enc_inputs, teacher_inputs], [dec_sequence_output])

In [ ]:
encoder_attention = build_encoder() 
seq2seq_attention = build_seq2seq_attention(encoder_attention, decoder_attention)
utils.plot_model(seq2seq_attention,
                 show_shapes=True,
                #show_dtype=True,
                show_layer_names=True,
                rankdir="TB",
                expand_nested=True,
                dpi=200,
                show_layer_activations=True,
                show_trainable=True)

In [ ]:
# On entraine le modèle
if not ((_MODEL_PATH / "encoder_attention.keras").exists() and (_MODEL_PATH / "decoder_attention.keras").exists() and (_MODEL_PATH / "seq2seq_attention.keras").exists()):
    seq2seq_attention.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    hist = seq2seq_attention.fit([X_train, teacher_train], y_train,
          validation_data=([X_val, teacher_val], y_val),
          batch_size=128,
          epochs=500,
          callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
    )
    pd.DataFrame.from_dict(hist.history).plot()
    encoder_attention.save(_MODEL_PATH / "encoder_attention.keras")
    decoder_attention.save(_MODEL_PATH / "decoder_attention.keras")
    seq2seq_attention.save(_MODEL_PATH / "seq2seq_attention.keras")
else:
    print("reload attentionnal model")
    encoder_attention = models.load_model(_MODEL_PATH / "encoder_attention.keras")
    decoder_attention = models.load_model(_MODEL_PATH / "decoder_attention.keras")
    seq2seq_attention = models.load_model(_MODEL_PATH / "seq2seq_attention.keras")

In [ ]:
# Pour prédire, on procède par étape comme pour le Seq2Seq mais il faut peut être adapter un peu l'algorithme pour inclure l'attention.
def predict_with_attention(seq2seq_attention, X):
    ...
    return y_pred, attention_matrix

In [ ]:
pred = predict_attention0(seq2seq_attention, X_test)
decode(X_test[0]), decode(y_test[0]), decode(pred[0])

In [ ]:
# On peut maintenant prédire - le score devrait être meilleur et être entre 80 et 90 %
score(y_test, pred)

### Attention matrix

Si la fonction de prédiction retourne la matrice d'attention, il est possible de l'afficher pour un exemple donné.

In [ ]:
def print_attention_matrix(id=0):
    """Inspect an individual sample.
    You can specify which dataset to examine with `ds_type`.
    """
    pred, attention_matrix = predict_attention(encoder_attention, decoder_attention, X_test)
    plt.imshow(attention_matrix[id], cmap='gray')
    plt.xticks(np.arange(len(y_test[id])), labels=[int_to_tok[int(i)] for i in y_test[id]])
    plt.yticks(np.arange(len(X_test[id])), labels=[int_to_tok[int(i)] for i in X_test[id]])

    print('Input:', decode(X_test[id]))
    print('Expected Output:', decode(y_test[id]))
    print('Predicted Output:', decode(pred[id]))

In [ ]:
# see what the post training output looks like
attn = print_attention_matrix(10)